In [15]:
import numpy as np
from scipy import signal

In [16]:
img = np.load("sample.npy")
img = np.floor(img/2)
filt = np.load("filter.npy")

# Defining the convolution

In [17]:
def conv2d(a, b):
    a = np.array(a)
    b = np.array(b)
    
    a_shape = np.shape(a)
    b_shape = np.shape(b)
    
    a_b_diff_dim_0 = np.abs(a_shape[0]-b.shape[0]+1)
    a_b_diff_dim_1 = np.abs(a_shape[1]-b.shape[1]+1)
    
    conv_res = np.zeros([a_b_diff_dim_0, a_b_diff_dim_1])
    
    for i in range(a_b_diff_dim_0):
        for j in range(a_b_diff_dim_1):
            conv_res[i,j] = np.sum(np.multiply(np.flip(b), a[i:i + b_shape[0], j:j + b_shape[1]]))
    
    return conv_res



def create_lookup_table():
    Multiplier = np.zeros((256, 256), dtype=int)
    for i in range(256):
        for j in range(256):
            Multiplier[i, j] = (i - 128) * (j - 128)
    return Multiplier


def My_Mult(a, b):
    # Ensure both matrices are of the same shape
    assert a.shape == b.shape, "Input matrices must have the same shape."

    # Convert the values of matrices a and b to integers
    a_int = a.astype(int)
    b_int = b.astype(int)

    # Add 128 to the integer values and use the value to look up the result in Multiplier
    result = Multiplier[a_int + 128, b_int + 128]

    return result



def My_conv2d(image, kernel):
    # Ensure the kernel is the same size as a region of the image
    assert kernel.shape[0] <= image.shape[0] and kernel.shape[1] <= image.shape[1], "Kernel size must be less than or equal to image size."

    # Initialize the result matrix
    result = np.zeros(image.shape, dtype=int)

    # Iterate over the image and apply convolution using My_Mult
    for i in range(image.shape[0] - kernel.shape[0] + 1):
        for j in range(image.shape[1] - kernel.shape[1] + 1):
            region = image[i:i + kernel.shape[0], j:j + kernel.shape[1]]
            result[i, j] = np.sum(My_Mult(region, kernel))

    return result

In [18]:
foo = conv2d(img, filt)

In [19]:
fofo = signal.convolve2d(img,filt, mode = "valid")

In [20]:
print(foo==fofo)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [21]:
Multiplier = create_lookup_table()

my_result = My_conv2d(img, filt)

error = np.sum(np.abs(foo - fofo))
print(f"Error between My_conv2d and conv2d: {error}")

ValueError: operands could not be broadcast together with shapes (256,256) (254,254) 